## Signal Processing

In [1]:
from pytomo3d.signal.process import process_stream as process
from obspy import read, read_inventory, read_events
from pytomo3d.adjoint.adjoint_source import calculate_adjsrc_on_stream
from pytomo3d.window.window import window_on_stream
from pycmt3d.source import CMTSource

Can not import pyasdf. ASDF not supported then


In [2]:
import pyflex

In [47]:
import pyadjoint

In [2]:
import pandas as pd

ImportError: No module named pandas

In [59]:
import numpy as np

In [4]:
from obspy.fdsn.client import Client
from obspy import UTCDateTime
client = Client("IRIS")

t = UTCDateTime("2014-09-01-00:00:00")
NN17 = client.get_stations(network="UW", station="NN17", level="response", starttime=t)#, endtime=t)
# stations = client https://discussions.apple.com/thread/3203909.get_stations(network="UW", station="NN", level="response", starttime=t)#, endtime=t)

/anaconda/envs/tomo_test/lib/python2.7/site-packages/obspy/__init__.py:160: ObsPyDeprecationWarning: Module 'obspy.fdsn' is deprecated and will stop working with the next ObsPy version. Please import module 'obspy.clients.fdsn' instead.
  ObsPyDeprecationWarning)
/anaconda/envs/tomo_test/lib/python2.7/site-packages/obspy/__init__.py:160: ObsPyDeprecationWarning: Module 'obspy.fdsn.client' is deprecated and will stop working with the next ObsPy version. Please import module 'obspy.clients.fdsn.client' instead.
  ObsPyDeprecationWarning)


using both pycmt3d and Obspy to read event info from CMT file

In [5]:
cmtfile = "NEGS_10_05_04/DATA/CMTSOLUTION"
cmtsource = CMTSource.from_CMTSOLUTION_file(cmtfile)
cmt_obspy = read_events(cmtfile)

Can not import pyasdf. ASDF not supported then


setting event variables, creating a pyflex event, writing event to a quakeml

In [6]:
cmt_obspy

1 Event(s) in Catalog:
2014-10-05T04:07:30.432000Z | +43.726, -121.310 | 1.7 mw

reading in variables from cmtsource, writing, then reading in with obspy

In [7]:
latitude = cmtsource.latitude
longitude = cmtsource.longitude
depth_in_m = cmtsource.depth_in_m
origin_time = cmtsource.origin_time
event = pyflex.Event(latitude, longitude, depth_in_m, origin_time)
cmt_obspy.write("event.xml", format="QUAKEML")
event_ml = read_events("event.xml")

/anaconda/envs/tomo_test/lib/python2.7/site-packages/obspy/core/event/base.pyc:378: UserWarning: The resource identifier 'smi:local/cmtsolution/20141005040730/moment_tensor' already exists and points to another object: 'MomentTensor(resource_id=ResourceIdentifier(id="smi:local/cmtsolution/20141005040730/moment_tensor"), derived_origin_id=ResourceIdentifier(id="smi:local/cmtsolution/20141005040730/origin#cmt"), moment_magnitude_id=ResourceIdentifier(id="smi:local/cmtsolution/20141005040730/magnitude#mw"), scalar_moment=441570510555.2226, tensor=Tensor(m_rr=370790000000.0, m_tt=88922000000.0, m_pp=-197980000000.0, m_rt=271800000000.0, m_rp=-126000000000.0, m_tp=-113750000000.0), source_time_function=SourceTimeFunction(type='triangle', duration=0.0))'.It will now point to the object referred to by the new resource identifier.
     warn_explicitt   UserWarning   __file__t   inspectt   currentframet   f_back   f_lineno(   R   Rw   Rf   (    (    s4   lib/python2.7/site-packages/obspy/

reading in waveforms, writing to mseed and reading back in for easier control of stats

In [8]:
obs = read(("NEGS_10_05_04/DATA/SAC/N*.EH?.UW.01"))

In [9]:
obs.write('obs.mseed', format='MSEED')
obs = read("obs.mseed")
stream = obs.copy()

set inventory to specific channel

In [10]:
obs[0].stats

         network: UW
         station: NM03
        location: 01
         channel: EHE
       starttime: 2014-10-05T04:07:14.000000Z
         endtime: 2014-10-05T04:08:39.996005Z
   sampling_rate: 249.999984741
           delta: 0.00400000024414
            npts: 21500
           calib: 1.0
         _format: MSEED
           mseed: AttribDict({'record_length': 4096, 'encoding': u'FLOAT32', 'filesize': 2830336, u'dataquality': u'D', 'number_of_records': 691, 'byteorder': u'>'})

In [11]:
# inv = read_inventory("NEGS_10_05_04/station_meta/fdsnws-station_2017-08-27T15_22_13Z.xml")
# inv = NN17.select(channel='*Z')
inv = NN17

adding event info to obs sac file

In [12]:
# sac_obs = obs[0].stats.sac
# sac_obs.evla = latitude
# sac_obs.evlo = longitude
# sac_obs.evdp = depth_in_m
# sac_obs.stla = inv
# obs[0].stats.sac = sac_obs

setting filter frequency band

In [13]:
# pre_filt = [.8, 1, 100, 120]
pre_filt = [1.6, 2, 30, 36]

processing observations as preparation for identical processing with synthetics, including removing instrument response

In [14]:
start_time = stream[2].stats.starttime # some traces start 2sec later
end_time = stream[2].stats.starttime + 30 # second

In [15]:
obs.detrend("linear")
obs.detrend("demean")
obs.taper(max_percentage=0.05, type="hann")

39 Trace(s) in Stream:

UW.NM03.01.EHE | 2014-10-05T04:07:14.000000Z - 2014-10-05T04:08:39.996005Z | 250.0 Hz, 21500 samples
...
(37 other traces)
...
UW.NN32.01.EHZ | 2014-10-05T04:07:16.000000Z - 2014-10-05T04:07:55.996002Z | 250.0 Hz, 10000 samples

[Use "print(Stream.__str__(extended=True))" to print all Traces]

We'll read a csv file of station information using pandas and create an inventory object for all stations using this DataFrame.

In [16]:
station_data = "MSA_Install_Data.csv"
stations = pd.read_csv(station_data)

In [17]:
stations

,NAME,Type,Longitude_,Latitude_W,Easting,Northing,Surface_El,Hole_Lock,Elevation_,_,Subsection,Road
0,NM22,surface,-121.308950,43.725225,636203.478346,4.842744e+06,5917.500000,4.0,5911.000000,1.802134,NaN,NaN
1,NM06,surface,-121.300140,43.724555,636914.604602,4.842685e+06,5971.000000,4.0,5979.000000,1.822866,NaN,NaN
2,NM42,surface,-121.279170,43.750590,638543.589159,4.845611e+06,6615.500000,4.0,6611.000000,2.015549,NaN,NaN
3,NM41,surface,-121.291205,43.710620,637666.190449,4.841152e+06,6143.500000,4.0,6158.000000,1.877439,NaN,NaN
4,NM08,surface,-121.302880,43.700835,636747.849486,4.840046e+06,6030.000000,4.0,6031.000000,1.838720,NaN,NaN
5,NM40,surface,-121.279640,43.718310,638580.190642,4.842025e+06,6461.000000,4.0,6457.000000,1.968598,NaN,NaN
6,NN07,borehole,-121.325087,43.751470,634844.825338,4.845633e+06,5543.000000,830.0,4703.000000,1.433841,NaN,NaN
7,NN19,borehole,-121.306950,43.718317,636380.242697,4.841980e+06,5914.000000,790.0,5102.000000,1.555488,SWSW of S28 T21S R12E,"USFS Road 600A-9735 (MP 11 ), Deschutes County..."
8,NN18,borehole,-121.302117,43.737577,636725.757453,4.844128e+06,6032.000000,810.0,5223.000000,1.592378,NESW of S21 T21S R12E,"USFS Road 685-9735 (MP 11 ), Deschutes County,..."
9,NN21,borehole,-121.289867,43.733180,637722.307837,4.843659e+06,6262.000000,690.0,5559.000000,1.694817,SESE of S21 T21S R12E,"USFS Road 687-9735 (MP 11 ), Deschutes County,..."


Let's convert feet to meters

In [18]:
surface_elevation = stations.Surface_El * .3048
depth = stations.Hole_Lock * .3048

In [19]:
from obspy import Inventory
from obspy.core.inventory import Network, Station, Channel

Let's initialize some inventory objects.

In [20]:
negs_inv = Inventory([], 'NEGS')

In [21]:
net = Network('UW',[])

In [22]:
response = NN17[0][0][0].response

Here we create a for loop through each station, create Channel objects for each component, create a Station object, append the channels we just created to that station, then append that station to the Network object we initialized before.

In [23]:
for i in range(len(stations)):
    cha_z = Channel('EHZ', '01',stations.Latitude_W[i], stations.Longitude_[i], surface_elevation[i], depth[i], 0, 90, response=response)
    cha_e = Channel('EHE', '01',stations.Latitude_W[i], stations.Longitude_[i], surface_elevation[i], depth[i], 90, 0, response=response)
    cha_n = Channel('EHN', '01',stations.Latitude_W[i], stations.Longitude_[i], surface_elevation[i], depth[i], 0, 0, response=response)
    sta_temp = Station(stations.NAME[i], stations.Latitude_W[i], stations.Longitude_[i], surface_elevation[i])
    sta_temp.channels.append(cha_z)
    sta_temp.channels.append(cha_e)
    sta_temp.channels.append(cha_n)
    net.stations.append(sta_temp)

In [24]:
negs_inv.networks.append(net)

A filter is applied then instrument response removed, 

In [25]:
new_obs = process(obs, remove_response_flag=True, inventory=negs_inv,
                  filter_flag=True, pre_filt=pre_filt)
# write out processed stream
# new_obs.write("test.mseed", format="MSEED")

#### matching observations and synthetics

In [26]:
synth_sac = read('NEGS_10_05_04/SEM/UW.N*.FX?.semd.sac.np')#, format = 'TSPAIR', startttime=-4.0000002E-03)
synth_start = synth_sac[0].stats.sac['b']
synth_sac.write("synth.mseed", format='MSEED')
synth = read("synth.mseed")
obs_data = new_obs.copy()

In [27]:
for i in range(len(synth)):
    synth_sac[i].stats.location = '01'

removing bad traces from stram

In [28]:
for tr in obs_data.select(station="NM08"):
    obs_data.remove(tr)
for tr in obs_data.select(station="NM41", channel="EHE"):
    obs_data.remove(tr)
for tr in synth.select(station="NM08"):
    synth.remove(tr)
for tr in synth.select(station="NM41", channel="FXE"):
    synth.remove(tr)

making stats compatible

In [29]:
# start_time = obs_data[0].stats.starttime
# origin = start_time + obs_data[0].stats.sac['o']
end_time = synth[0].stats.endtime
for i in range(len(synth)):
    synth[i].stats.starttime = origin_time - synth_start
#     end_time = synth[i].stats.endtime
# synth[0].stats.endtime = end_time

padding zeros before start of synth

In [30]:
for i in range(len(synth)):
    synth[i].trim(starttime=start_time, endtime=end_time, pad=True, fill_value=0)
sampling_rate = 1/synth[0].stats.delta
npts = synth[0].stats.npts

interpolating obs

In [31]:
for i in range(len(obs_data)):
    obs_data[i].interpolate(sampling_rate, starttime=start_time, npts=npts)

pre-processing

In [32]:
min_frequency = 2.0
# max_frequency = 28
max_frequency = 30

In [33]:
synth.detrend("linear")
synth.detrend("demean")
synth.taper(max_percentage=0.05, type="hann")
synth.filter("bandpass", freqmin=min_frequency, freqmax=max_frequency,
                  corners=4, zerophase=True)

69 Trace(s) in Stream:

UW.NM03..FXE | 2014-10-05T04:07:16.000000Z - 2014-10-05T04:07:44.427600Z | 2500.0 Hz, 71070 samples
...
(67 other traces)
...
UW.NN32..FXZ | 2014-10-05T04:07:16.000000Z - 2014-10-05T04:07:44.427600Z | 2500.0 Hz, 71070 samples

[Use "print(Stream.__str__(extended=True))" to print all Traces]

In [34]:
obs_data.detrend("linear")
obs_data.detrend("demean")
obs_data.taper(max_percentage=0.05, type="hann")
# obs_data.filter("bandpass", freqmin=min_frequency, freqmax=max_frequency, corners=4, zerophase=True)

35 Trace(s) in Stream:

UW.NM03.01.EHE | 2014-10-05T04:07:16.000000Z - 2014-10-05T04:07:44.427600Z | 2500.0 Hz, 71070 samples
...
(33 other traces)
...
UW.NN32.01.EHZ | 2014-10-05T04:07:16.000000Z - 2014-10-05T04:07:44.427600Z | 2500.0 Hz, 71070 samples

[Use "print(Stream.__str__(extended=True))" to print all Traces]

plotting to check

#### stats for synth

In [35]:
# synth[.stats.network = obs_data[0].stats.network
# synth[0].stats.station = obs_data[0].stats.station
# synth[0].stats.location = obs_data[0].stats.location
# synth[0].stats.channel = 'FXZ'

#### adding event info

## Window Selection

In [36]:
min_period=1/30.0
max_period=1/2.0
config = pyflex.Config(
    min_period=min_period, max_period=max_period,
    stalta_waterlevel=0.1, tshift_acceptance_level=0.25,
    dlna_acceptance_level=10.0, cc_acceptance_level=0.80,
    c_0=0.6, c_1=1.0, c_2=0, c_3a=1.0, c_3b=1.0, c_4a=0.5, c_4b=0.5)

In [37]:
config_dict = {
    "Z": config,
#     "R": pyflex.Config,
#     "T": pyflex.Config
}

In [38]:
obs_scaled = obs_data.copy()
for i in range(len(obs_data)):
    obs_scaled[i].data = obs_scaled[i].data * 10

In [39]:
# both = obs_scaled
# both += synth 

# both.plot(starttime=origin_time, endtime=origin_time + 4)

Now we create a window object for each trace pair, looping over each trace in the observed data and selecting the corresponding trace of the synthetic data for comparison.

In [40]:
# synth.select(station=obs_scaled[0].stats.station, channel=('*' + obs_scaled[0].stats.channel[-1]))

1 Trace(s) in Stream:
UW.NM03..FXE | 2014-10-05T04:07:16.000000Z - 2014-10-05T04:07:44.427600Z | 2500.0 Hz, 71070 samples

In [41]:
# negs_inv.select(station=trace.stats.station, channel=('*' + trace.stats.channel[-1]))

NameError: name 'trace' is not defined

In [42]:
windows = []
for trace in obs_scaled:
    station=trace.stats.station
    channel=('*' + trace.stats.channel[-1])
    inv = negs_inv.select(station=station, channel=channel)
    window = pyflex.select_windows(trace, synth.select(station=station, channel=channel), config, event=event_ml, station=inv, plot=False)
    windows.append(window)

/Users/stephensoerens/tomo_test/pyflex/src/pyflex/window_selector.py:623: ObsPyDeprecationWarning: The getTravelTimes() function is deprecated. Please use the obspy.taup.TauPyModel class directly.
  model=self.config.earth_model)


In [43]:
windows

[[Window(left=40160, right=40331, center=40245, channel_id=UW.NM03.01.EHE, max_cc_value=0.90845316072, cc_shift=-73, dlnA=2.60130086417),
  Window(left=40687, right=40871, center=40779, channel_id=UW.NM03.01.EHE, max_cc_value=0.809988089648, cc_shift=-66, dlnA=0.6646119899),
  Window(left=41171, right=41363, center=41267, channel_id=UW.NM03.01.EHE, max_cc_value=0.80684793962, cc_shift=33, dlnA=1.81099994058),
  Window(left=43227, right=43426, center=43326, channel_id=UW.NM03.01.EHE, max_cc_value=0.880190770801, cc_shift=-15, dlnA=1.63964587821),
  Window(left=44378, right=44551, center=44464, channel_id=UW.NM03.01.EHE, max_cc_value=0.81739041051, cc_shift=-14, dlnA=2.10007198751),
  Window(left=61293, right=61494, center=61393, channel_id=UW.NM03.01.EHE, max_cc_value=0.850949499171, cc_shift=22, dlnA=2.10579710814),
  Window(left=65095, right=65265, center=65180, channel_id=UW.NM03.01.EHE, max_cc_value=0.948294536061, cc_shift=-2, dlnA=3.28602164045)],
 [Window(left=40287, right=40469,

In [44]:
# window = window_on_stream(
#     obs_scaled, synth, config_dict, station=inv, event=event_ml, figure_mode=False)

In [45]:
# adjoint_source = pyadjoint.calculate_adjoint_source(
#     adj_src_type="waveform_misfit",
#     observeed=obs_scaled, 
#     synthetic=synth, 
#     min_period = 1/30.0, max_period = 1/2.0,
#     left_window_border=

In [48]:
config_multitaper = pyadjoint.ConfigMultiTaper(
    min_period, max_period,
    lnpt=15,
    transfunc_waterlevel=1.0E-10,
    water_threshold=0.02,
    ipower_costaper=10,
    min_cycle_in_window=0.5,
    taper_type='hann',
    taper_percentage=0.3,
    mt_nw=4.0,
    num_taper=5,
    dt_fac=2.0,
    phase_step=1.5,
    err_fac=2.5,
    dt_max_scale=3.5,
    measure_type='dt',
    dt_sigma_min=1.0,
    dlna_sigma_min=0.5,
    use_cc_error=False,
    use_mt_error=False)

In [49]:
config_cc = pyadjoint.ConfigCrossCorrelation(
    min_period, max_period,
#     lnpt=15,
#     transfunc_waterlevel=1.0E-10,
#     water_threshold=0.02,
#     ipower_costaper=10,
#     min_cycle_in_window=0.5,
    taper_type='hann',
    taper_percentage=0.3,
#     mt_nw=4.0,
#     num_taper=5,
#     dt_fac=2.0,
#     phase_step=1.5,
#     err_fac=2.5,
#     dt_max_scale=3.5,
    measure_type='dt',
    dt_sigma_min=1.0,
    dlna_sigma_min=0.5,
    use_cc_error=True)
#     use_mt_error=False)

In [50]:
# windows[0:2].left
# windows_time = windows.left * sampling_rate, windows.ri
# window = windows[0]

In [51]:
# print(pyadjoint.calculate_adjoint_source(
#     "multitaper_misfit", 
#     obs_scaled,
#     synth,
#     windows,
#     config,
#     adjoint_src=False))

In [60]:
left_window_border = windows[0][0].left
right_window_border = windows[0][0].right
deltat = synth[0].stats.delta

# left_sample = int(np.floor(left_window_border / deltat)) + 1
# nlen = int(np.floor((right_window_border - left_window_border) / deltat)) + 1
# right_sample = left_sample + nlen

left_sample = left_window_border
right_sample = right_window_border
nlen = right_sample - left_sample

d = np.zeros(nlen)
s = np.zeros(nlen)

In [67]:
windows[0][0]

Window(left=40160, right=40331, center=40245, channel_id=UW.NM03.01.EHE, max_cc_value=0.90845316072, cc_shift=-73, dlnA=2.60130086417)

In [66]:
adjsrc = pyadjoint.adjoint_source.calculate_adjoint_source(
#     "multitaper_misfit",
    "cc_traveltime_misfit",
    obs_scaled[0], synth[0], config_cc, windows[0][0])

TypeError: 'Window' object is not iterable

In [ ]:
print(adjsrc)

In [ ]:
# adjsrcs = pytomo3d.adjoint.adjsrc.calculate_adjsrc_on_stream(
#     obs_scaled, synth, windows, config, 'multitaper_misfit',
#     figure_mode=True, figure_dir="./figure")

## Generic Tests

In [ ]:
obs, syn = pyadjoint.utils.get_example_data()